# Appendix: Creating a Scene in python

A pre-defined sky scene can be injested into MIRISim, although the flexibility exists for the user to create their own custom scene.  This notebook shows the steps required to create a viable (albeit simple) scene for use within mirisim. The equivalent code placed in a .ini file is given in parallel with the python definitions.

This example creates a scene with two objects: a star, and an extended galaxy. Both have imported spectral energy distributions (SEDs) and a few additional spectral lines overlaid.

In [1]:
%env PYSYN_CDBS=/Users/niklasviebig/Desktop/Python/JWST_semester_project/trds


env: PYSYN_CDBS=/Users/niklasviebig/Desktop/Python/JWST_semester_project/trds


In [2]:
## skysim is the module within MIRISim that creates scenes.

from mirisim.skysim import Background, sed, kinetics,Galaxy, Point, Skycube
from mirisim.skysim import wrap_pysynphot as wS
import numpy as np


### setup the scene

There is a short preamble required for a .ini file to be executable. There is no python equivalent, so it is mentioned here briefly before delving into the python aspects of creating a scene

    [sky]
        name = scene1
        loglevel = 1

## Creating a Galaxy

The first target to be created in this simple sky scene is a galaxy.  The steps to creating a complete galaxy include:

1. setting up the spatial extent of the galaxy
1. creating an SED to apply to the extended emission
1. Add the SED to the Galaxy
1. add a line of sight velocity distribution to the SED

#### Step 1: Setup the spatial extent of the galaxy

The galaxy created within python below has the same properties as this parameter grouping expressed in a .ini file::

    [Galaxy]
        Type       = Galaxy                 # Type of target
        Cen        = -2. 2.                  # RA,DEC (offset, specified in arcseconds)
        n          = 1.                     # Sersic index of the Galaxy
        re         = 0.1                    # Effective radius (arcsec)
        q          = 0.5                    # Axial ratio
        pa         = 15.                    # position angle (deg)

In [3]:
galaxy = Galaxy(Cen=(-2.,2.),n=1.,re=0.1,q=0.5,pa=15.)

2022-04-04 15:36:21,148 - INFO - Initializing Galaxy
2022-04-04 15:36:21,154 - INFO - Initializing Galaxy
2022-04-04 15:36:21,157 - INFO - Initializing Galaxy


#### Step 2: Create a spectral energy distribution for the Galaxy

Here we show an example of importing a pysynphot SED. The parameters required as input to a pysynphot SED can be found on the pysynphot website. note that the `sedname` must be consistent with one provided in the pysynphot directories, the location of which was set when mirisim was installed. To find it:

    echo $PYSYN_CDBS
    

The .ini file equivalent of creating this pysynphot SED in python is as follows. Note that when placed in a .ini file, this set of properties can be indented for clarity.

    [[sed]]
        Type       = pysynphot              # specify SED from pySynPhot librarires
        family     = bkmodels               # SED Family from which to draw.
        sedname    = bk_b0005               # Template within calatlogue to be used
        wref       = 10.                    # Reference wavelength for scaling (in micron)
        flux       = 1E+3                   # Reference flux for scaling (in microJy)
        
        
The 'bkmodels' are the Buser-Kurucz stellar atmosphere models, more details of which can be found at [this link](http://www.stsci.edu/hst/observatory/crds/bkmodels.html).  This model were chosen for this example because the SED extends to 20 micron. Most of the SEDs in the pysynphot repository do not reach MIRI wavelengths.


In [4]:
# specify which of the pysynphot models to use (in a dictionary)
PYSPsedDict = {'family':'bkmodels','sedname':'bk_b0004','flux':1E+3,'wref':10.}
# read that dictionary into the pysynphot interpreter
sedE = wS.PYSPSed(**PYSPsedDict)



#### Step 3: Add the SED to the galaxy object

The  SED component can now be added to the properties of the galaxy created in Step 1.  This step has no equivalent in the .ini file, as this is done automatically by placing the SEDs underneath the instance of `[Galaxy]`

In [5]:

# link the SED to the galaxy object
galaxy.set_SED(sedE)

#### Step 4: add a velocity map to the Galaxy

The template galaxy created here is expected to be rotating, which will shift the SED as a function of position within the galaxy. This is accounted for by applying a velocity map to the galaxy object.  The .ini file equivalent of the python code below is:

      [[velomap]]
            Type       = FlatDisk        # Type of Velocity map to initialise
            Cen        = -2. 2.          # Specify the centre of the disk 
            vrot       = 200.            # Rotational Velocity (km/s)
            pa         = 15.             # Position angle of the velocity map (deg)
            q          = 0.5             # Axial ratio of major and minor axes
            c          = 0.              # measure of diskiness/boxiness 
            
Here, we've created a flattened galatic disk. Note that many of the parameters set here are identical to those set when initialising the galaxy at the beginning.



In [6]:
# Create a dictionary of parameters
VMAPpars = {'vrot':200.,'Cen':(-2.,2.),'pa':15.,'q':0.5,'c':0}
# create an instance of a flattened disk
velomap = kinetics.FlatDisk(**VMAPpars)


# apply the velocity mapping to the galaxy

galaxy.set_velomap(velomap)

2022-04-04 15:36:27,052 - INFO - Initializing FlatDisk


## Creating a "Star"

Because a star is a point source, the inputs are a little simpler. An SED can (and should) still be specified (for this example a star is a simple blackbody), however there is no point in specifying a velocity distribution since all of the flux is coming from a single point.

The steps involved in creating a star here include:

1. initialising a point source
1. adding a blackbody spectrum

The equivalent in a .ini file would entail:

    [Star]
        Type = Point
        Cen = 0. 0.
        vel = 0.0

        [[sed]]
            Type = BB
            Temp = 1e4
            flux = 100.
            wref = 10.


In [7]:
## 1. initialise the point source spatial position
star = Point(Cen=(0.,0.))
## 2a. create a dictionary with the required parameters for a blackbody
BBparams = {'Temp':1e4,'wref':10.,'flux':100.}
## 2b. create the blackbody spectrum
BlackBody = sed.BBSed(**BBparams)
## 2c. add that spectrum to the 'star'
star.set_SED(BlackBody)

2022-04-04 15:36:28,465 - INFO - Initializing Point


## Creating a Scene from a star and a Galaxy

The two components of the scene can now simply be added together to form a scene.  To this two component scene, we will also (below) add some background radiation. The equivalent .ini file syntax for adding the background is:

    [Background]
        level = low
        gradient = 5.
        pa = 45.
        centreFOV = 0 0

After completing the scene, a text description can be output to the screen using the print function


In [8]:
# create a background object
bg = Background(level='low',gradient=5.,pa=45.)

# sum the background,galaxy and star
scene = bg + galaxy + star

## print the text version to the screen
#print(scene)


2022-04-04 15:36:29,729 - INFO - Initializing Background


## Exporting the scene to a FITS file

With the scene created, there's now the oportunity to write it out to a FITS file.  To do this requires additional specifications, including:

    * Field of View (FOV)
    * Spatial Sampling (spatsampling)
    * Output Wavelength Range (wrange)
    * Wavelength Sampling (wsampling)
    * Time (time)

Note that the last parameter, time, needs to be specified for time dependent backgrounds only (which are not yet implemented).


In [9]:
# The field of view and its sampling are specified in arcsecondsy
FOV = np.array([[-3,3],[-4,4]])  # 6x8" field of view
spatsampling = 0.1               # 0.1" spatial sampling (resolution)

# The wavelength range is specified in microns
wrange = [5.,15.]           # the output only covers 5 to 15 microns 
wsampling = 0.002           # 0.002 micron wavelength resolution

# Because time dependent backgrounds are not yet implemented, 
# 'time' does not need to be set


With the additional parameters pertaining to the properties of the output FITS file set, the scene created above can be output to a FITS file

**NOTE:** The option <code> overwrite = False </code> ensures that any FITS file in the current directory with the suggested output file name exists, it will not be overwritten. To change this, set <code> overwrite = True </code>

In [ ]:
scene.writecube(cubefits = 'scene.FITS',
                FOV = FOV, spatsampling = spatsampling, 
                wrange = wrange, wsampling = wsampling,
                overwrite = True, time = 0.0)

In [ ]:
def show_outputs(MIRISim_outputdir,output_type):
    '''
    plot the specified channel of the MIRISim outputs
    :param MIRISim_outputdir:  
        name of the date-labelled dir. holding the MIRISIM outputs
    :param output_type: 
        type of output to process 
        (e.g. illum_models, det_images or skycubes)
    '''
    
    infits = glob.glob('{}/{}/*.fits'.format(MIRISim_outputdir,output_type))[0]
    
    hdulist = fits.open(infits)
    

    hdu_index = 1
    if len(hdulist[hdu_index].data.shape) > 3:
        integ,frames,nx,ny = hdulist[hdu_index].data.shape
        image = hdulist[hdu_index].data[integ-1,frames-1,:,:]
        
        
    else:
        image = hdulist[hdu_index].data[0,:,:]
              
    norm = colors.LogNorm(image.mean() + 0.5 * image.std(), image.max(), clip=True)   
    plt.imshow(image,origin = 'lower', cmap = cm.viridis,interpolation = 'nearest',norm = norm)
    plt.title('{}'.format(infits.split('/')[-1]))
    plt.xlabel("'RA' Direction")
    plt.ylabel("'DEC' Direction")


    plt.colorbar()

